In [60]:
import os
import pandas as pd

import pandasql as psql

# hub
h_user = pd.read_parquet("/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hubs/hub_user.parquet")
h_user_play_session = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hubs/hub_user_play_session.parquet"
)
h_user_payment_detail = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hubs/hub_user_payment_detail.parquet"
)
# hub_satellite

hs_user_details = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hub_satellite/user/hub_user.parquet"
)
hs_user_registartion_details = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hub_satellite/user_registration/hub_user_registration.parquet"
)
hs_user_play_session = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hub_satellite/user_play_session/sat_user_play_session_details.parquet"
)
hs_user_plan = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hub_satellite/user_plan/sat_user_plan_details.parquet"
)
hs_user_payment_details = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/hub_satellite/user_payment_detail/sat_user_payment_detail_details.parquet"
)

# link
l_user_user_payment = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/links/user_registration_plan/link_user_registration_plan.parquet"
)

l_user_user_play_channel = pd.read_parquet(
    "/home/sagemaker-user/dice_model_project/Project-/output/raw_vault/links/user_play_session/link_user_play_session.parquet"
)

In [26]:
query = """
select count(*), channel_code   
FROM hs_user_play_session group by  channel_code 
"""

result = psql.sqldf(query, globals())
result.head()

,count(*),channel_code
0,941,BROWSER
1,931,MOBILE


In [56]:
query = """
select count(distinct a.user_id)  as 'onetime_user'
FROM h_user a 
inner join l_user_user_payment b on
b.user_id = a.user_id
 inner join h_user_payment_detail c 
 on c.payment_detail_id	 = b.payment_detail_id	
 inner join hs_user_plan d on d.hash_key = c.hash_key
where payment_frequency_code = 'ONETIME'
"""

result = psql.sqldf(query, globals())
result.head()

,onetime_user
0,75


In [57]:
query = """
select count(distinct a.user_id)  as 'subscribed_user'
FROM h_user a 
inner join l_user_user_payment b on
b.user_id = a.user_id
 inner join h_user_payment_detail c 
 on c.payment_detail_id	 = b.payment_detail_id	
 inner join hs_user_plan d on d.hash_key = c.hash_key
where payment_frequency_code != 'ONETIME'
"""

result = psql.sqldf(query, globals())
result.head()

,subscribed_user
0,140


In [66]:
query = """SELECT 
    SUM(
        CASE d.payment_frequency_code
            WHEN 'ANNUALLY' THEN 9.99
            WHEN 'ONETIME' THEN 14.99
            WHEN 'MONTHLY' THEN 1.99
            ELSE 0
        END
    ) AS total_payment_amount
FROM h_user a
INNER JOIN l_user_user_payment b 
    ON b.user_id = a.user_id
INNER JOIN h_user_payment_detail c 
    ON c.payment_detail_id = b.payment_detail_id
INNER JOIN hs_user_plan d 
    ON d.hash_key = c.hash_key
INNER JOIN hs_user_payment_details e 
    ON a.hash_key = e.hash_key;



"""

result = psql.sqldf(query, globals())
result.head()

,total_payment_amount
0,863.05
